In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [4]:
# fixed parameters
hyper_epoch = 50
threshold = -1.15
reuse = False
block = MEGNet_block
head_mol,head_atom,head_edge = head_mol,head_atom,head_edge
epochs = 50
batch_size = 32
clip = 2
lr = 1e-4
epochs_type = epochs

In [ ]:
for _ in range(hyper_epoch):
    # model parameters

    head = np.random.choice([feedforwardHead_Update,feedforwardCombineHead_Update],p=[0.6,0.4])
    data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle',\
                             '../Data/{}_data_SOAP_expand_PCA_otherInfo.pickle',\
                             '../Data/{}_data_atomInfo_otherInfo.pickle'])
    logLoss = np.random.choice([True,False])
    weight = np.random.rand()*3 if logLoss else np.random.rand()*0.75
    dim = int(np.random.choice([128,256,512]))
    layer1 = int(np.random.choice([3,4,5]))
    layer2 = int(np.random.choice([3,4,5]))
    factor = int(np.random.choice([2,3,4]))
    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [head,data,dim,logLoss,weight,layer1,layer2,factor]])))

    train_dl,val_dl = get_data(data,batch_size)
    model = GNN_multiHead(reuse,block,head,head_mol,head_atom,head_edge,\
                          dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
    
    model,train_loss_perType,val_loss_perType = train_type_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,\
                                                                    scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)
    if model is not None:
        save_results2(train_loss_list,val_loss_list,head,data,dim,logLoss,weight,layer1,layer2,factor)
        save_model_type2(bestWeight,opt,head,data,dim,logLoss,weight,layer1,layer2,factor)

In [6]:
# fixed parameters
hyper_epoch = 50
threshold = -1.15
reuse = False
block = MEGNet_block
head_mol,head_atom,head_edge = head_mol,head_atom,head_edge
epochs = 1
batch_size = 32
clip = 2
lr = 1e-4
epochs_type = epochs

In [ ]:
for _ in range(hyper_epoch):
    # model parameters

    head = np.random.choice([feedforwardHead_Update,feedforwardCombineHead_Update],p=[0.6,0.4])
    data = '../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle'
    logLoss = np.random.choice([True,False])
    weight = np.random.rand()*3 if logLoss else np.random.rand()*0.75
    dim = int(np.random.choice([128,256,512]))
    layer1 = int(np.random.choice([3,4,5]))
    layer2 = int(np.random.choice([3,4,5]))
    factor = int(np.random.choice([2,3,4]))
    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [head,data,dim,logLoss,weight,layer1,layer2,factor]])))

    train_dl,val_dl = get_data(data,batch_size)
    model = GNN_multiHead(reuse,block,head,head_mol,head_atom,head_edge,\
                          dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
    
    model,train_loss_perType,val_loss_perType,bestWeight = train_type_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,\
                                                                    scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)
    if model is not None:
        save_results2(train_loss_perType,val_loss_perType,head,data,dim,logLoss,weight,layer1,layer2,factor)
        save_model_type2(bestWeight,opt,head,data,dim,logLoss,weight,layer1,layer2,factor)


training on <class 'functions_refactor.feedforwardHead_Update'>__data_ACSF_expand_PCA_otherInfo.pickle_128_True_0.2831571741916332_3_3_3



In [9]:
def fun(a,*c):
    return c

In [10]:
fun(1,2,3,4,5)

(2, 3, 4, 5)

In [12]:
save_results2(train_loss_list,val_loss_list,head,data,dim,logLoss,weight,layer1,layer2,factor)

NameError: name 'train_loss_list' is not defined

In [13]:
val_loss_perType

NameError: name 'val_loss_perType' is not defined